In [ ]:
!pip install ucimlrepo

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf

In [ ]:
rice_cammeo_and_osmancik = fetch_ucirepo(id=545)

In [ ]:
X = rice_cammeo_and_osmancik.data.features
y = rice_cammeo_and_osmancik.data.targets

In [ ]:
X.info()
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3810 entries, 0 to 3809
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Area               3810 non-null   int64  
 1   Perimeter          3810 non-null   float64
 2   Major_Axis_Length  3810 non-null   float64
 3   Minor_Axis_Length  3810 non-null   float64
 4   Eccentricity       3810 non-null   float64
 5   Convex_Area        3810 non-null   int64  
 6   Extent             3810 non-null   float64
dtypes: float64(5), int64(2)
memory usage: 208.5 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3810 entries, 0 to 3809
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Class   3810 non-null   object
dtypes: object(1)
memory usage: 29.9+ KB


In [ ]:
X.describe()

,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Eccentricity,Convex_Area,Extent
count,3810.000000,3810.000000,3810.000000,3810.000000,3810.000000,3810.000000,3810.000000
mean,12667.727559,454.239180,188.776222,86.313750,0.886871,12952.496850,0.661934
std,1732.367706,35.597081,17.448679,5.729817,0.020818,1776.972042,0.077239
min,7551.000000,359.100006,145.264465,59.532406,0.777233,7723.000000,0.497413
25%,11370.500000,426.144752,174.353855,82.731695,0.872402,11626.250000,0.598862
50%,12421.500000,448.852493,185.810059,86.434647,0.889050,12706.500000,0.645361
75%,13950.000000,483.683746,203.550438,90.143677,0.902588,14284.000000,0.726562
max,18913.000000,548.445984,239.010498,107.542450,0.948007,19099.000000,0.861050


In [ ]:
X.isnull().sum()

Area                 0
Perimeter            0
Major_Axis_Length    0
Minor_Axis_Length    0
Eccentricity         0
Convex_Area          0
Extent               0
dtype: int64

In [ ]:
y['Class'].value_counts()

Osmancik    2180
Cammeo      1630
Name: Class, dtype: int64

In [ ]:
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)
X_normalized

array([[0.67593733, 0.87923163, 0.90121587, ..., 0.88801055, 0.69391702,
        0.20757716],
       [0.62533005, 0.71409491, 0.64808716, ..., 0.69197987, 0.64600914,
        0.32456423],
       [0.62339377, 0.75006612, 0.73434911, ..., 0.78984635, 0.63563643,
        0.53857594],
       ...,
       [0.43002992, 0.44045819, 0.40864083, ..., 0.60847407, 0.43266526,
        0.35564534],
       [0.20515754, 0.17532451, 0.16992207, ..., 0.56901705, 0.20868495,
        0.44453926],
       [0.34175321, 0.24088172, 0.16869838, ..., 0.28376388, 0.34001406,
        0.84022485]])

In [ ]:
label_encoder = LabelEncoder()
y['Class'] = label_encoder.fit_transform(y['Class'])

<ipython-input-32-d20a001ba4a5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['Class'] = label_encoder.fit_transform(y['Class'])


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

In [ ]:
model = Sequential() # Initialising the ANN/DNN
model.add(Dense(units = 64, activation = 'relu', input_shape=(X_train.shape[1],)))
model.add(Dense(units = 16, activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))

In [ ]:
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy',
              metrics=[
                  tf.keras.metrics.Precision(name='precision'),
                  tf.keras.metrics.Recall(name='recall'),
                  'accuracy'
                  ])


In [ ]:
history = model.fit(X_train.astype('float'), y_train, batch_size = 1, epochs = 10, validation_split=0.2, verbose = 0)

In [ ]:
yhat_test1 = model.predict(X_test.astype(float))
yhat_test1 = [ 1 if y>=0.5 else 0 for y in yhat_test1 ]
print(classification_report(y_test,yhat_test1))

test_loss = model.evaluate(X_test, y_test)
print(f"Test loss: {test_loss}")

# Extracting training and validation loss and accuracies
training_loss = history.history['loss'][0]
validation_loss = history.history['val_loss'][0]
training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

print(f"training_loss: {training_loss}")
print(f"validation_loss: {validation_loss}")
print(f"training_accuracy: {training_accuracy}")
print(f"validation_accuracy: {validation_accuracy}")

24/24 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.92      0.92      0.92       350
           1       0.93      0.93      0.93       412

    accuracy                           0.93       762
   macro avg       0.93      0.93      0.93       762
weighted avg       0.93      0.93      0.93       762

24/24 [==============================] - 0s 2ms/step - loss: 0.1883 - precision: 0.9322 - recall: 0.9345 - accuracy: 0.9278
Test loss: [0.18834932148456573, 0.9322034120559692, 0.9344660043716431, 0.9278215169906616]
training_loss: [0.38356196880340576, 0.22034424543380737, 0.22422100603580475, 0.22206194698810577, 0.22703798115253448, 0.22520610690116882, 0.22898998856544495, 0.23029226064682007, 0.22987166047096252, 0.2310105264186859]
validation_loss: [0.21520565450191498, 0.2429259568452835, 0.2049793154001236, 0.2279914915561676, 0.23665504157543182, 0.20973365008831024, 0.21275149285793304, 0.22692976891994

## Where does your model fit in the fitting graph.
Accuracy, Precision, Recall, and F1-score are consistently high on the test dataset (around 93%), indicating that the model performs well on unseen data.

Loss metrics show that the validation loss (with values ranging from about 0.20 to 0.24) is reasonably close to the training loss (which decreases to about 0.23 by the end of training), and the test loss is also low (0.1883).

We can see the training, validating, and testing metrics perform good and similar to each other which show that our model is just right. No overfitting or underfitting happen.

## What are the next 2 models you are thinking of and why?
1. SVM: SVM is a good model for classication, and we are trying to classify rice base on features, which is appropriate.
2. Naive Bayes: Naive Bayes is a good model for classication task, and we are trying to classify rice base on features, which is appropriate.

## Conclusion section: What is the conclusion of your 1st model? What can be done to possibly improve it?

The model currently perform well, with above 90% accuracy on testing data. However, there still some room to improve. We can tune our hyperparameter to try to achieve better metrics.